In [ ]:
import openai
import os
import json
from typing import Dict, Any, List, Optional, Union

# ------------------------------
# STEP 0: Configure OpenAI client
# ------------------------------
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4o"  # Use GPT-4o for tool calling

# ------------------------------
# STEP 1: Define tool schemas
# ------------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_nutrition_info",
            "description": "Get nutritional information for a specific food item",
            "parameters": {
                "type": "object",
                "properties": {
                    "food_name": {
                        "type": "string",
                        "description": "The name of the food item to get nutritional information for"
                    },
                    "serving_size": {
                        "type": "string",
                        "description": "The serving size to calculate nutrition for (e.g., '100g', '1 cup')"
                    }
                },
                "required": ["food_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_daily_intake",
            "description": "Calculate if a meal meets daily nutritional requirements",
            "parameters": {
                "type": "object",
                "properties": {
                    "meal_items": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "List of food items in the meal"
                    },
                    "user_profile": {
                        "type": "string",
                        "description": "User profile type (e.g., 'adult_male', 'adult_female', 'child')"
                    }
                },
                "required": ["meal_items", "user_profile"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "find_alternatives",
            "description": "Find alternative foods with similar nutritional profiles",
            "parameters": {
                "type": "object",
                "properties": {
                    "food_name": {
                        "type": "string",
                        "description": "The food to find alternatives for"
                    },
                    "dietary_restriction": {
                        "type": "string",
                        "description": "Dietary restriction to consider (e.g., 'vegan', 'gluten-free', 'low-carb')",
                        "enum": ["vegan", "vegetarian", "gluten-free", "low-carb", "low-fat", "low-sodium", "none"]
                    }
                },
                "required": ["food_name"]
            }
        }
    }
]

# ------------------------------
# STEP 2: Tool handler implementations
# ------------------------------
def get_nutrition_info(food_name: str, serving_size: str = "100g") -> str:
    # In a real implementation, this would query a nutrition database
    nutrition_database = {
        "apple": {"calories": 52, "protein": 0.3, "carbs": 14, "fat": 0.2, "fiber": 2.4},
        "chicken breast": {"calories": 165, "protein": 31, "carbs": 0, "fat": 3.6, "fiber": 0},
        "rice": {"calories": 130, "protein": 2.7, "carbs": 28, "fat": 0.3, "fiber": 0.4},
        "spinach": {"calories": 23, "protein": 2.9, "carbs": 3.6, "fat": 0.4, "fiber": 2.2},
    }

    if food_name.lower() in nutrition_database:
        nutrients = nutrition_database[food_name.lower()]
        return (f"Nutritional information for {food_name} ({serving_size}):\n"
                f"Calories: {nutrients['calories']} kcal\n"
                f"Protein: {nutrients['protein']}g\n"
                f"Carbohydrates: {nutrients['carbs']}g\n"
                f"Fat: {nutrients['fat']}g\n"
                f"Fiber: {nutrients['fiber']}g")
    else:
        return f"Nutritional information for '{food_name}' not found in database."

def calculate_daily_intake(meal_items: List[str], user_profile: str) -> str:
    # In a real implementation, this would calculate nutritional totals and compare to RDAs
    rda = {
        "adult_male": {"calories": 2500, "protein": 56, "carbs": 310, "fat": 78},
        "adult_female": {"calories": 2000, "protein": 46, "carbs": 250, "fat": 62},
        "child": {"calories": 1800, "protein": 34, "carbs": 225, "fat": 56}
    }

    # Simplified implementation
    meal_str = ", ".join(meal_items)
    if user_profile in rda:
        profile_rda = rda[user_profile]
        return (f"Analysis for meal ({meal_str}) based on {user_profile} profile:\n"
                f"This meal provides approximately 35% of your daily caloric needs.\n"
                f"It covers 40% of protein, 30% of carbs, and 25% of fat requirements.\n"
                f"Recommended daily values for {user_profile}:\n"
                f"Calories: {profile_rda['calories']} kcal, Protein: {profile_rda['protein']}g, "
                f"Carbs: {profile_rda['carbs']}g, Fat: {profile_rda['fat']}g")
    else:
        return f"Profile '{user_profile}' not recognized. Available profiles: adult_male, adult_female, child"

def find_alternatives(food_name: str, dietary_restriction: str = "none") -> str:
    # In a real implementation, this would query a database of alternative foods
    alternatives = {
        "beef": {
            "vegan": ["tofu", "seitan", "tempeh"],
            "vegetarian": ["eggs", "tofu", "tempeh"],
            "low-carb": ["chicken", "turkey", "pork"]
        },
        "milk": {
            "vegan": ["almond milk", "soy milk", "oat milk"],
            "low-fat": ["skim milk", "almond milk", "cashew milk"]
        },
        "rice": {
            "low-carb": ["cauliflower rice", "quinoa", "bulgur"],
            "gluten-free": ["quinoa", "buckwheat", "millet"]
        }
    }

    if food_name.lower() in alternatives:
        food_alternatives = alternatives[food_name.lower()]
        if dietary_restriction in food_alternatives:
            alt_list = ", ".join(food_alternatives[dietary_restriction])
            return f"Alternatives to {food_name} suitable for {dietary_restriction} diets: {alt_list}"
        else:
            return f"No specific {dietary_restriction} alternatives found for {food_name}."
    else:
        return f"No alternatives found for '{food_name}' in database."

# ------------------------------
# STEP 3: Tool registry and router
# ------------------------------
tool_registry = {
    "get_nutrition_info": get_nutrition_info,
    "calculate_daily_intake": calculate_daily_intake,
    "find_alternatives": find_alternatives,
}

def execute_tool_call(tool_call) -> str:
    """Execute a tool call from the OpenAI API response"""
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)

    if function_name in tool_registry:
        try:
            return tool_registry[function_name](**function_args)
        except Exception as e:
            return f"[ERROR] Failed to execute '{function_name}': {str(e)}"
    else:
        return f"[ERROR] Unknown tool call: '{function_name}'"

# ------------------------------
# STEP 4: Main agent function
# ------------------------------
def run_agent(user_prompt: str, context: Dict[str, Any] = None) -> str:
    """
    Process user request using OpenAI's tool calling capabilities

    Args:
        user_prompt: The user's request
        context: Additional context to provide to the model

    Returns:
        The final response to the user
    """
    messages = [
        {
            "role": "system",
            "content": "You are a nutrition assistant that helps users understand the nutritional content of foods, "
                       "calculate daily intake requirements, and find alternative foods that match dietary restrictions."
        }
    ]

    # Add context to the prompt if provided
    if context:
        context_str = json.dumps(context, indent=2)
        messages.append({
            "role": "system",
            "content": f"Additional context:\n{context_str}"
        })

    # Add user message
    messages.append({
        "role": "user",
        "content": user_prompt
    })

    # Initial call to the model
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        temperature=0.2
    )

    assistant_message = response.choices[0].message
    messages.append(assistant_message)

    # Check if the model wants to call a tool
    if assistant_message.tool_calls:
        # Execute each tool call
        for tool_call in assistant_message.tool_calls:
            # Call the appropriate function
            function_response = execute_tool_call(tool_call)

            # Add the function response to the messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": tool_call.function.name,
                "content": function_response,
            })

        # Get a new response from the model with the function results
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.2
        )

        return second_response.choices[0].message.content
    else:
        # The model chose to respond directly
        return assistant_message.content

# ------------------------------
# Example usage
# ------------------------------
if __name__ == "__main__":
    # Simulated user request examples
    examples = [
        "What's the nutritional content of an apple?",
        "Is a meal with chicken breast, rice, and spinach enough for my daily needs as an adult male?",
        "I'm vegan. What can I eat instead of beef?"
    ]

    for example in examples:
        print(f"\nUser query: {example}")
        result = run_agent(example)
        print("\nAgent Response:")
        print(result)
        print("-" * 50)